In [1]:
import glob
print(glob.glob("/share/*"))

['/share/input.txt', '/share/us-stock-dataset', '/share/Transformers1']


In [2]:
import sys
import os

import torch
from torch import nn as nn
import glob

from t4.timeseries_transformer import TransformerConfig, TransformerRunner

from timeseries.csv_reader import read_and_merge_csv_files

from t3_karpathy.commons.commons import BaseTransformerConfig
from t3_karpathy.token_codec import TokenCodec
from t4.generic_dataloader import GenericDataloader
from timeseries.timeseries_transformer import TimeseriesDataloader



In [3]:
stocks_to_load = [
    "AAPL", "TSLA",
    "A", "GOOG", "AMZN", "PYPL", "NVDA", "AMD",
    "NFLX", "MSFT", "INTC", "CSCO", "ADBE", "CRM", "QCOM", "TXN", "AVGO",
    "INTU", "ORCL", "COST", "SBUX", "AMGN", "CHTR", "GILD", "CMCSA", "BKNG",
    "MDLZ", "FISV", "BIIB", "MU", "MCD", "AMAT", "ADP", "ILMN", "ATVI", "ISRG",
    "ADSK", "LRCX", "BIDU", "JD", "REGN", "WBA", "VRTX", "KHC", "WMT", "ZM", "MELI",
    "TMUS", "CTSH", "XLNX", "PCAR", "ALGN", "WDAY", "SIRI", "CTXS", "ADI", "EXC", "LULU",
    "MAR", "KLAC", "PAYX", "EA", "ILMN", "ALXN", "MNST", "BMRN", "EBAY", "CTAS", "VRSK",
    "IDXX", "CDNS", "NXPI", "ASML", "INCY", "KLAC", "MCHP", "SNPS", "SWKS", "VRSN",
    "WDC", "WYNN", "XLNX", "ZBRA", "ZTS", "AEP", "AIG", "ALL", "AXP", "BA", "BAC",
    "BK", "BLK", "C", "CAT", "CL", "COF", "COP", "COST", "CSCO", "CVS", "CVX",
    "DD", "DHR", "DIS", "DOW", "DUK", "EMR", "EXC", "F", "FDX", "GD", "GE", "GILD",
    "GM", "GOOG", "GOOGL", "GS", "HD", "HON", "IBM", "INTC", "JNJ", "JPM", "KHC", "KMI",
    "KO", "LLY", "LMT", "LOW", "MA", "MCD", "MDLZ", "MDT", "MET", "MMM",
    'BLND', 'BLNG', 'BLNKW', 'BLNGU', 'BLNGW', 'BLNK', 'BLPH', 'BLRX',
    'BLTE', 'BLU', 'BLUA', 'BLUE', 'BLW', 'BLX', 'BLZE', 'BMA', 'BMAC', 'BMAQ', 'BMAQR', 'BMAQU'
]

directory_path = '/share/us-stock-dataset/Data/Stocks'

# stocks_to_load = [s.split("\\")[1].replace(".csv", "") for s in glob.glob(directory_path + "//*.csv")]
#
# stock_number_to_load = 200
#
# stocks_to_load = stocks_to_load[1000:1000+stock_number_to_load]

dataloader = TimeseriesDataloader(directory_path, stocks_to_load, add_diff=False)

print(dataloader.get_data().shape)

Reading and merging CSV files: ['AAPL', 'TSLA', 'A', 'GOOG', 'AMZN', 'PYPL', 'NVDA', 'AMD', 'NFLX', 'MSFT', 'INTC', 'CSCO', 'ADBE', 'CRM', 'QCOM', 'TXN', 'AVGO', 'INTU', 'ORCL', 'COST', 'SBUX', 'AMGN', 'CHTR', 'GILD', 'CMCSA', 'BKNG', 'MDLZ', 'FISV', 'BIIB', 'MU', 'MCD', 'AMAT', 'ADP', 'ILMN', 'ATVI', 'ISRG', 'ADSK', 'LRCX', 'BIDU', 'JD', 'REGN', 'WBA', 'VRTX', 'KHC', 'WMT', 'ZM', 'MELI', 'TMUS', 'CTSH', 'XLNX', 'PCAR', 'ALGN', 'WDAY', 'SIRI', 'CTXS', 'ADI', 'EXC', 'LULU', 'MAR', 'KLAC', 'PAYX', 'EA', 'ILMN', 'ALXN', 'MNST', 'BMRN', 'EBAY', 'CTAS', 'VRSK', 'IDXX', 'CDNS', 'NXPI', 'ASML', 'INCY', 'KLAC', 'MCHP', 'SNPS', 'SWKS', 'VRSN', 'WDC', 'WYNN', 'XLNX', 'ZBRA', 'ZTS', 'AEP', 'AIG', 'ALL', 'AXP', 'BA', 'BAC', 'BK', 'BLK', 'C', 'CAT', 'CL', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'EXC', 'F', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD'

In [4]:
dataloader.get_data()[0]

tensor([8.4947e-01, 1.5927e+00, 4.3930e+01, 2.4991e+00, 4.4688e+00, 3.6710e+01,
        8.9489e-01, 1.5500e+01, 1.1964e+00, 3.6282e+01, 2.5214e+01, 3.7781e+01,
        1.6275e+01, 4.3000e+00, 5.9089e+01, 3.4013e+01, 1.1836e+01, 2.7018e+01,
        2.4375e+01, 3.0857e+01, 2.4508e+00, 4.6329e+01, 3.5000e+01, 1.3459e+00,
        1.1592e+01, 3.0750e+02, 1.2123e+01, 6.3542e+00, 3.1083e+01, 3.7509e+01,
        2.2424e+01, 2.3480e+01, 2.5348e+01, 1.9586e+01, 1.2144e+00, 2.0278e+00,
        8.0529e+00, 3.2735e+01, 1.2254e+01, 2.0084e+01, 1.2312e+01, 1.8248e+01,
        1.8781e+01, 5.2814e+01, 4.3873e+01, 6.2000e+01, 2.7616e+01, 4.5404e+01,
        1.8462e+00, 2.6263e+00, 1.6875e+01, 4.8690e+01, 3.6274e+01, 2.8982e+01,
        5.1753e+00, 1.4000e+01, 1.1493e+01, 2.9200e+01, 1.3783e+01, 2.4923e+01,
        1.9586e+01, 4.4922e-02, 1.2750e+01, 6.9658e+00, 2.6095e+01, 2.6463e+01,
        4.0156e+00, 2.2062e+01, 1.3085e+01, 2.7219e+01, 4.4828e+01, 2.9200e+01,
        5.8364e+00, 3.2781e+01, 2.7434e+

In [4]:
device = 'cuda'

torch.manual_seed(1337)

config = TransformerConfig(None, precision=torch.bfloat16, batch_size=32, block_size=16, n_embed=64, n_head=4, n_layer=4)

runner = TransformerRunner(config, dataloader.get_data())

runner.train_iterate_n(2500)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
# print(dataloader.token_codec.decode(runner.generate(context, max_new_tokens=2000)[0].tolist()))

0.201472 M parameters


RuntimeError: mat1 and mat2 must have the same dtype